##Connect to database! 
###Get Neighborhood Data

In [2]:
import pandas as pd
import json
import psycopg2
import random
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict
import numpy as np

###Get tweets wtihin california, and shapes with the county of SF ('075')

In [6]:
conn_dict = {'dbname':'zipfiantwitter', 'user':'clwilloughby', 'password': '', 'host':'/tmp'}
conn = psycopg2.connect(dbname=conn_dict['dbname'], user=conn_dict['user'], host='/tmp')
cur = conn.cursor()

map_query = """
        SELECT
                DISTINCT
                ST_asGEOJSON(wkb_geometry) geometry
                , geoid10

                FROM sf_neighb 
                WHERE countyfp10 = '075'
                ;"""

geodf = pd.read_sql(map_query, conn)
#geodf.to_csv('sf_only_sql_shapes.csv')

###format the geo pd

In [74]:
geodf['geoid10'] = geodf.geoid10.astype('int64')
#set the index as the geoid
geodf.set_index('geoid10', inplace=True)

###get tweet counts dataframe

two options. One is tweets perminute. One is ust all tweets. 

In [3]:
df_tweet_count = pd.read_csv('../data_pipeline/data/tweet_p_m_withgeoid.csv')
#df_tweet_count = pd.read_csv('tweets_with_time_wk2.csv')

In [4]:
#this column always pops up...
df_tweet_count.drop('Unnamed: 0', 1, inplace=True)
#set the df as the index
df_tweet_count.set_index('geoid10', inplace=True)

###Merge SF Geometry with SF Tweets
Two methods. Sometimes They work or don't work.

In [87]:
test = df_tweet_count.join(geodf, rsuffix='_new')
test.to_csv('all_tpmin_with_coordinates.csv')

In [83]:
map_df = pd.concat([geodf, sfdf], axis = 1)

ValueError: Shape of passed values is (8, 23499), indices imply (8, 7386)

In [77]:
#also if there are no tweets, fill with zero.
map_df = map_df[map_df.geometry.notnull()]
#fill empty counts with zero (no tweets)
#map_df['count'].fillna(0, inplace = True)
map_df.reset_index(inplace=True)

ValueError: Shape of passed values is (8, 23499), indices imply (8, 7386)

In [8]:
map_test = pd.read_csv('all_tpmin_with_coordinates.csv')

###Get a Random Color For Each Neighborhood
####Later will encode information by color and hue

In [281]:
def getrandom_color(df, color_file):
    '''takes in a dataframe of gemoetry and geoid, and a list of hex colors
    Creates a new colomn of shape colors. Returns the new dataframe.'''
    #import the colors csv
    cdf = pd.read_csv('colors.csv')
    #make it a list
    colors = cdf[' color'].tolist() 
    num_col = df.shape[0] 
    df['hexcolor'] = [random.choice(colors) for i in xrange(num_col)]
    return df

In [222]:
df_col = getrandom_color(map_df, 'colors.csv')


/Users/christy/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [223]:
df_col.head()


,geoid10,geometry,count,text,hexcolors
0,60750101001000,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,NaN,#CC0099
1,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",9,"[u'tfw', u'internet', u'archiv', u'sudden', u'...",#CCCCFF
2,60750101001002,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,NaN,#3333CC
3,60750101001003,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,NaN,#FF3399
4,60750101001004,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",6,"[u'deep', u'blue', u'adobelif', u'hearth', u'a...",#3333CC


###A. Convert DataFrame of Neighborhoods to JsonFile for Mapping

In [253]:
def add_properties_geo(row):
    geoid = row['geoid10']
    hexcolor = row['hexcolor']
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),  "properties": {'geoid': geoid , 'hexcolor' : hexcolor }}
    return geo_json

list_to_export = [add_properties_geo(row) for idx, row in df_col.iterrows()]

with open('sf_geo.json', 'w') as outfile:
    json.dump(list_to_export, outfile)

KeyError: 'hexcolor'

###B. Convert DataFrame of Conuts to Json File for Mapping

In [9]:
def add_properties_geo(row):
    geoid = row['geoid10']
    hexcolor = row['hexcolor']
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),  "properties": {'geoid': geoid , 'hexcolor' : hexcolor }}
    return geo_json

def dataframe_to_geojson(df, outfilename):
    '''Takes in a dataframe with a count and geoid10. Dumps it into a json geojason file'''
    df['geoid10'] = df['geoid10'].astype('str')
    
    list_to_export = [add_properties_geo(row) for idx, row in map_df.iterrows()]
    with open(outfilename, 'w') as outfile:
        json.dump(list_to_export, outfile)

In [21]:
float(len(map_df['count'].unique()))

30.0

In [23]:
map_df['count'].unique()

array([  0.00000000e+00,   4.20000000e+01,   1.80000000e+01,
         1.00000000e+00,   2.00000000e+00,   3.00000000e+00,
         5.00000000e+00,   4.00000000e+00,   6.00000000e+00,
         1.00000000e+01,   2.60000000e+01,   7.00000000e+00,
         8.00000000e+00,   3.90000000e+01,   2.40000000e+01,
         1.20000000e+01,   9.40000000e+01,   9.00000000e+00,
         1.40000000e+01,   2.00000000e+01,   1.90000000e+01,
         4.50000000e+01,   1.70000000e+01,   1.60000000e+01,
         1.10000000e+01,   1.30000000e+01,   3.63000000e+02,
         2.70000000e+01,   1.79000000e+02,   1.95000000e+03])

####Manually heat map colors

In [26]:
def fill_colors(df):
    '''Bulky way to create a manual heat map based on the counts of values.
    Need to find an automated way.
    Takes a dataframe with a COUNT column. Uses to return an added COLOR column
    which contains a spectrum of red colors.
    '''
    #create a dictionary of what color goes with each unique value

    values= float(len(df['count'].unique()))

    color_list = ['#665229','#806633','#997A3D','#B28F47','#CCA352','#E6B85C','#FFD175','#FFE0A3','#FFF0D1','#FFFAF0',"F0F8FF"]
    value_dict = defaultdict(str)
    for index, numb in enumerate(df['count'].unique()):
        color= round(index/values*10)
        numb= int(numb)
        value_dict[numb] = color_list[int(color)]
    df['hexcolor'] = df['count'].map(value_dict.get)
    return df


In [27]:
df_color =fill_colors(map_df)
df_color = df_color[['geoid10', 'geometry', 'hexcolor']]
dataframe_to_geojson(df_color, 'data/heatmap_geo.json')

/Users/christy/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


###NEw Test. Get Rate of tweets as a variable in geogjson

In [11]:
map_test.head()

,Unnamed: 0,geoid10,date,hour,min,min_count,count,t_p_min,geometry
0,0,60750101001001,2015-07-15,6,11,1,1,1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122...."
1,1,60750101001001,2015-07-15,8,10,1,1,1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122...."
2,2,60750101001001,2015-07-15,19,59,1,1,1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122...."
3,3,60750101001001,2015-07-15,20,0,1,1,1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122...."
4,4,60750101001001,2015-07-25,4,90,2,2,1,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122...."


In [14]:
#Now have tweets per hour, and better count
map_test['hr_cnt'] = 1
map_test = map_test.groupby(['geoid10', 'geometry', 'hour']).agg(sum).reset_index()

In [52]:
map_test.head()

,geoid10,geometry,hour,Unnamed: 0,min,min_count,count,t_p_min,hr_cnt,tph
0,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,409,270,10,10,6,6,1.666667
1,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",1,446,281,10,10,7,7,1.428571
2,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",2,412,320,11,11,7,7,1.571429
3,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",3,249,144,7,7,6,6,1.166667
4,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",4,281,422,12,12,5,5,2.400000


In [17]:
map_test['tph'] = map_test['count']/map_test['hr_cnt']


In [18]:
map_test.head()

,geoid10,geometry,hour,Unnamed: 0,min,min_count,count,t_p_min,hr_cnt,tph
0,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,409,270,10,10,6,6,1.666667
1,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",1,446,281,10,10,7,7,1.428571
2,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",2,412,320,11,11,7,7,1.571429
3,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",3,249,144,7,7,6,6,1.166667
4,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",4,281,422,12,12,5,5,2.400000


In [53]:
keydf= pd.read_csv('geo_key_cloud.csv')

In [54]:
image_key = keydf[['geoid10']]

In [55]:
image_key = image_key.reset_index().rename(columns = {'index':'origional_index'})

In [59]:
image_key.tail()

,origional_index,geoid10
1180,1180,60759809001083
1181,1181,60759809001090
1182,1182,60759809001091
1183,1183,60759809001109
1184,1184,60759809001127


In [27]:
#dont need below: both are integers
#image_key.geoid10 = image_key.geoid10.apply(lambda x: str(x))


In [79]:
test = map_test.merge(image_key)
test.head()

,geoid10,geometry,hour,Unnamed: 0,min,min_count,count,t_p_min,hr_cnt,tph,origional_index
0,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,409,270,10,10,6,6,1.666667,0
1,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",1,446,281,10,10,7,7,1.428571,0
2,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",2,412,320,11,11,7,7,1.571429,0
3,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",3,249,144,7,7,6,6,1.166667,0
4,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",4,281,422,12,12,5,5,2.400000,0


In [43]:
test.columns

Index([u'geoid10', u'geometry', u'hour', u'Unnamed: 0', u'min', u'min_count',
       u'count', u't_p_min', u'hr_cnt', u'tph', u'origional_index'],
      dtype='object')

In [80]:
test = test.groupby(['geoid10', 'origional_index', 'geometry']  ).agg(sum)
test = test.reset_index()

In [86]:
test['count']

'190'

In [173]:
def add_properties_geo(row):
    geoid = row['geoid10']
    tweetrate = row['tpm']
    image = row['image']
    tweetcount = row['count']
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),  "properties": {'image': image, 'geoid': geoid ,'tweetrate': tweetrate }}
    return geo_json

def dataframe_to_geojson(df, outfilename):
    '''Takes in a dataframe with a count and geoid10. Dumps it into a json geojason file'''
    df['geoid10'] = df['geoid10'].astype('str')
    df["count"] = df['tpm'].astype('str')
    df["count"] = df['count'].astype('str')
    list_to_export = [add_properties_geo(row) for idx, row in df.iterrows()]
    with open(outfilename, 'w') as outfile:
        json.dump(list_to_export, outfile)

In [176]:
dataframe_to_geojson(total_tweet, 'test_wordcloud/data/geo_rate_test.json')

In [ ]:
'http://s357.photobucket.com/user/clwillerbee/media/'0.png.html

In [82]:
def add_properties_geo(row, imgvalue):
    geoid = row['geoid10']
    tweetrate = row['count']
    image = imgvalue
    tweetcount = imgvalue
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),  "properties": {'image': imgvalue, 'geoid': geoid ,'tweetrate': tweetrate }}
    return geo_json

def dataframe_to_geojson(df, outfilename):
    '''Takes in a dataframe with a count and geoid10. Dumps it into a json geojason file'''
    df['geoid10'] = df['geoid10'].astype('str')
    df["count"] = df['count'].astype('str')
    list_to_export = []
    for idx, row in test.iterrows():
        imgvalue = 'http://i357.photobucket.com/albums/oo11/clwillerbee/' + str(idx) + '.png'
        list_to_export.append(add_properties_geo(row, imgvalue))
    with open(outfilename, 'w') as outfile:
        json.dump(list_to_export, outfile)

In [83]:
dataframe_to_geojson(test, 'wordcloud/data/geo_wordcloud.json')

In [89]:
test['count'].astype(int).unique()

array([  190,   158,     1,    88,   332,    69,    78,     3,     2,
           5,   151,     4,    77,   145,    82,    79,   150,   134,
          10,   182,    65,   127,    75,   152,     8,   206,    68,
          59,    71,    62,    84,    80,    85,     6,    86,    72,
          70,    76,   448,    24,    66,    12,   456,   126,   147,
         161,    81,    87,    74,   274,    73,    98,   377,   683,
         153,    67,   223,    83,   571,   362,   515,    64,    58,
         163,   142,    14,   138,    56,   169,    17,    91,   729,
          60,    16,    11,    13,   139,    93,   200,  5776,     9,
         279,   162,     7,   287,   141,    57,   144,    63,   230,
         496,   137,   135,   264,   207,   292,   326,    18,   148,
         291,   140,    53,   154,   222, 12749,   214])

In [73]:
for idx, row in test[:10].iterrows():
        imgvalue = 'http://i357.photobucket.com/albums/oo11/clwillerbee/' + str(idx) + '.png'
        print imgvalue

http://i357.photobucket.com/albums/oo11/clwillerbee/0.png
http://i357.photobucket.com/albums/oo11/clwillerbee/1.png
http://i357.photobucket.com/albums/oo11/clwillerbee/2.png
http://i357.photobucket.com/albums/oo11/clwillerbee/3.png
http://i357.photobucket.com/albums/oo11/clwillerbee/4.png
http://i357.photobucket.com/albums/oo11/clwillerbee/5.png
http://i357.photobucket.com/albums/oo11/clwillerbee/6.png
http://i357.photobucket.com/albums/oo11/clwillerbee/7.png
http://i357.photobucket.com/albums/oo11/clwillerbee/8.png
http://i357.photobucket.com/albums/oo11/clwillerbee/9.png


In [ ]:
<img src="http://i357.photobucket.com/albums/oo11/clwillerbee/999.png" border="0" alt=" photo 999.png"/></a>